In [6]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
load_dotenv()
GROQ_API_KEY =os.environ.get("GROQ_API_KEY")

In [6]:
llm =ChatGroq(model="llama-3.3-70b-versatile")
llm.invoke("what is ai")

AIMessage(content="Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as:\n\n1. **Learning**: AI systems can learn from data and improve their performance over time.\n2. **Problem-solving**: AI systems can analyze problems and find solutions.\n3. **Reasoning**: AI systems can draw inferences and make decisions based on available data.\n4. **Perception**: AI systems can interpret and understand data from sensors, such as images, speech, and text.\n\nThe goal of AI is to create systems that can think and act like humans, or even surpass human capabilities in specific domains. AI has many applications, including:\n\n1. **Virtual assistants**: AI-powered virtual assistants, such as Siri, Alexa, and Google Assistant, can understand voice commands and perform tasks.\n2. **Image recognition**: AI-powered image recognition systems can identify objects, people, and patterns in images.\n3. **Natural La

# Pydanic class
If we want the model to return a Pydantic object, we just need to pass in the desired Pydantic class. The key advantage of using Pydantic is that the model-generated output will be validated. Pydantic will raise an error if any required fields are missing or if any fields are of the wrong type.

In [ ]:
from pydantic import BaseModel,Field
from typing import Optional

class Base(BaseModel): #It automatically validates types and raises errors for invalid data.
    name:str
    model_provider:str
    prompt :str
    chat:Optional[str]

# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    A: str = Field(description="The A of the joke")
    B: str = Field(description="The B to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about dog")

Joke(A='Why did the dog go to the vet?', B='Because he was feeling ruff!', rating=8)

# TypedDict or JSON Schema
If you don't want to use Pydantic, explicitly don't want validation of the arguments, or want to be able to stream the model outputs, you can define your schema using a TypedDict class. We can optionally use a special Annotated

In [10]:
from typing import Optional
from typing_extensions import Annotated, TypedDict


# TypedDict
class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]

    # Alternatively, we could have specified setup as:

    # setup: str                    # no default, no description
    # setup: Annotated[str, ...]    # no default, no description
    # setup: Annotated[str, "foo"]  # default, no description

    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")


{'setup': 'Why did the cat join a band?',
 'punchline': 'Because it wanted to be the purr-cussionist',
 'rating': 8}

# Tools Call

In [21]:
from pydantic import BaseModel,Field
from langchain_core.output_parsers import PydanticToolsParser

class add(BaseModel):
    """Add the numbers"""
    a:int 
    b:int

class multi(BaseModel):
    """Multi the numbers"""
    a:int
    b:int

tool=[add,multi]

llm_tool=llm.bind_tools(tool)
chain=llm_tool|PydanticToolsParser(tools=[add,multi])

chain.invoke("What is  add 3 + 4")

[add(a=3, b=4)]

# chache 
use to reduce the llm calls

In [ ]:
# It fetch the repeated question from llm chache
from langchain_core.globals import set_llm_cache
from langchain_core.caches import InMemoryCache
set_llm_cache(InMemoryCache)

In [4]:
# We can do the same thing with a SQLite cache
from langchain_community.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.db"))